In [0]:
Use GPU

In [2]:
import os
import pickle
import numpy as np
import pandas as pd
from keras.utils import np_utils
from collections import defaultdict
from keras.models import Sequential, Model
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import mean_absolute_error
from keras.preprocessing.sequence import skipgrams
from keras.layers.merge import Dot, multiply, concatenate
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Flatten, Dropout

Using TensorFlow backend.


In [0]:
upload 
movies.csv
ratings.csv
From Avenu2Learn (week 6)

In [0]:
# read data
movie_titles = pd.read_csv(
    'movies.csv',
    usecols=['movieId', 'title'],
    dtype={'movieId': 'int16', 'title': 'str'})

df_user= pd.read_csv(
    'ratings.csv',
    usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

Each record (row) shows the rating for a user, item (movie) pair.

In [7]:
movie_titles.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [9]:
df_user.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
len(df_user.userId.unique()), len(df_user.movieId.unique())

(610, 9724)

unique number between (0, #users) to each user and do the same for movies.

In [0]:
df_user.userId = df_user.userId.astype('category').cat.codes.values
df_user.movieId = df_user.movieId.astype('category').cat.codes.values

In [12]:
df_user.head()

,userId,movieId,rating
0,0,0,4.0
1,0,2,4.0
2,0,5,4.0
3,0,43,5.0
4,0,46,5.0


# Generate train and test

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_user, test_size=0.3)

In [14]:
train.head()

,userId,movieId,rating
36457,248,909,4.5
13104,83,120,4.0
31818,218,3490,3.5
95069,598,9454,3.0
78872,488,3409,0.5


In [0]:
n_users, n_movies = len(df_user.userId.unique()), len(df_user.movieId.unique())

# ---Model 1----

In [18]:
dim_embedddings = 30
bias = 3


w_inputs = Input(shape=(1,), dtype='int32')
u_inputs = Input(shape=(1,), dtype='int32')

w = Embedding(n_movies+1, dim_embedddings, name="work")(w_inputs)
u = Embedding(n_users+1, dim_embedddings, name="user")(u_inputs)

o = multiply([w, u])
o = Dropout(0.5)(o)
o = Flatten()(o)
o = Dense(1)(o)

rec_model = Model(inputs=[w_inputs, u_inputs], outputs=o)
rec_model.compile(loss='mae', optimizer='adam', metrics=["mae"])
rec_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
work (Embedding)                (None, 1, 30)        291750      input_1[0][0]                    
__________________________________________________________________________________________________
user (Embedding)     

In [19]:
history = rec_model.fit([train.movieId, train.userId], train.rating, nb_epoch=3,validation_split=0.2, verbose=2)

predictions = rec_model.predict([test.movieId, test.userId])
test_performance = mean_absolute_error(test.rating, predictions)

print(" Test Mae model 1 : %s " % test_performance)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 56468 samples, validate on 14117 samples
Epoch 1/3
 - 8s - loss: 2.5201 - mean_absolute_error: 2.5201 - val_loss: 1.2369 - val_mean_absolute_error: 1.2369
Epoch 2/3
 - 7s - loss: 0.9336 - mean_absolute_error: 0.9336 - val_loss: 0.7720 - val_mean_absolute_error: 0.7720
Epoch 3/3
 - 7s - loss: 0.7266 - mean_absolute_error: 0.7266 - val_loss: 0.7217 - val_mean_absolute_error: 0.7217
 Test Mae model 1 : 0.7146915 


# ----Model 2---

In [20]:
dim_embedddings = 30
bias = 1

w_inputs = Input(shape=(1,), dtype='int32')
u_inputs = Input(shape=(1,), dtype='int32')

w = Embedding(n_movies+1, dim_embedddings, name="work")(w_inputs)
w_bis = Embedding(n_movies+1, bias, name="workbias")(w_inputs)

u = Embedding(n_users+1, dim_embedddings, name="user")(u_inputs)
u_bis = Embedding(n_users+1, bias, name="userbias")(u_inputs)

o = multiply([w, u])
o = concatenate([o, u_bis, w_bis])
o = Dropout(0.5)(o)
o = Flatten()(o)
o = Dense(1)(o)

rec_model = Model(inputs=[w_inputs, u_inputs], outputs=o)
rec_model.compile(loss='mae', optimizer='adam', metrics=["mae"])

rec_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
work (Embedding)                (None, 1, 30)        291750      input_3[0][0]                    
__________________________________________________________________________________________________
user (Embedding)                (None, 1, 30)        18330       input_4[0][0]                    
__________________________________________________________________________________________________
multiply_2

In [21]:
history = rec_model.fit([train.movieId, train.userId], train.rating, nb_epoch=3,validation_split=0.2, verbose=2)
predictions = rec_model.predict([test.movieId, test.userId])
test_performance = mean_absolute_error(test.rating, predictions)
print(" Test Mae model 1 : %s " % test_performance)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 56468 samples, validate on 14117 samples
Epoch 1/3
 - 8s - loss: 2.2473 - mean_absolute_error: 2.2473 - val_loss: 1.0389 - val_mean_absolute_error: 1.0389
Epoch 2/3
 - 8s - loss: 0.9841 - mean_absolute_error: 0.9841 - val_loss: 0.7604 - val_mean_absolute_error: 0.7604
Epoch 3/3
 - 9s - loss: 0.7440 - mean_absolute_error: 0.7440 - val_loss: 0.7118 - val_mean_absolute_error: 0.7118
 Test Mae model 1 : 0.7032975 


# ----Model 3----

In [0]:
dim_embedddings = 30
bias = 1

# inputs
w_inputs = Input(shape=(1,), dtype='int32')
u_inputs = Input(shape=(1,), dtype='int32')

w = Embedding(n_movies+1, dim_embedddings, name="work")(w_inputs)
w_bis = Embedding(n_movies+1, bias, name="workbias")(w_inputs)

u = Embedding(n_users+1, dim_embedddings, name="user")(u_inputs)
u_bis = Embedding(n_users+1, bias, name="userbias")(u_inputs)

o = multiply([w, u])
o = Dropout(0.5)(o)
o = concatenate([o, u_bis, w_bis])

o = Flatten()(o)
o = Dense(10, activation="relu")(o)
o = Dense(1)(o)

rec_model = Model(inputs=[w_inputs, u_inputs], outputs=o)
rec_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
input_12 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
work (Embedding)                 (None, 1, 30)         660930      input_11[0][0]                   
____________________________________________________________________________________________________
user (Embedding)                 (None, 1, 30)         302190      input_12[0][0]                   
___________________________________________________________________________________________

In [0]:
history = rec_model.fit([train.movieId, train.userId], train.rating, nb_epoch=3,validation_split=0.2, verbose=2)
predictions = rec_model.predict([test.movieId, test.userId])
test_performance = mean_absolute_error(test.rating, predictions)
print(" Test Mae model 1 : %s " % test_performance)